In [2]:
import os
import requests
import pandas as pd
import math
import logging
import time

# Parameters
csv_file = "seamounts.csv"  # Path to the .csv file
output_folder = "seamounts_galore"  # Folder to save downloaded images
annotated_folder = "seamounts_annotated"  # Folder to save annotated images
bbox_folder = "seamounts_bboxes"  # New folder to save bounding box images
bbox_file = "seamounts_bboxes.csv"  # File to save bounding box data
tile_pixels = 800  # Width and height of the image in pixels
api_base_url = "https://www.gmrt.org/services/ImageServer"
max_retries = 3  # Max retries for failed downloads
txt_file = "missing in pixel list.txt"  # File containing PEAKID's to download

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ensure output folders exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(annotated_folder, exist_ok=True)
os.makedirs(bbox_folder, exist_ok=True)

# Load PEAKIDs from the txt file
try:
    # Remove '.0' from PEAKID in the .txt file and keep them as strings
    with open(txt_file, "r") as f:
        peakids_to_download = {line.strip().replace(".png", "").replace(".0", "") for line in f.readlines()}
    logging.info(f"Loaded {len(peakids_to_download)} PEAKIDs to download from {txt_file}.")
except Exception as e:
    logging.error(f"Error loading .txt file: {e}")
    exit()

# Load CSV file and filter rows with LAT between -60 and 60
try:
    df = pd.read_csv(csv_file)
    df = df[(df["LAT"] >= -60) & (df["LAT"] <= 60)]  # Filter latitudes between -60 and 60

    # Ensure PEAKID in the CSV is a string for consistent comparison
    df["PEAKID"] = df["PEAKID"].astype(str)

    # Debug: Check PEAKIDs in the CSV file
    logging.debug("First few PEAKIDs from CSV file:")
    logging.debug(df["PEAKID"].head().to_list())

    # Filter by PEAKIDs
    df = df[df["PEAKID"].isin(peakids_to_download)]  # Filter by PEAKIDs
    logging.info(f"Filtered {len(df)} records from {csv_file}.")

    # Debug: Check the filtered PEAKIDs
    logging.debug("First few filtered PEAKIDs from CSV file:")
    logging.debug(df["PEAKID"].head().to_list())
except Exception as e:
    logging.error(f"Error loading .csv file: {e}")
    exit()

# Prepare bounding box file
bbox_data = []

# Process each record
for index, row in df.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            # Extract data from the row
            file_name = str(row["PEAKID"])  # Use PEAKID as the file name
            center_lon = float(row["LONG"])  # Longitude from LONG
            center_lat = float(row["LAT"])  # Latitude from LAT
            area_km2 = float(row["AREA2D"])  # Area in km^2

            # Calculate the radius in degrees from the area
            radius_deg = math.sqrt(area_km2 / math.pi) / 111.0  # 1 degree ≈ 111 km

            # Calculate bounding box coordinates
            x_min = center_lon - radius_deg
            x_max = center_lon + radius_deg
            y_min = center_lat - radius_deg
            y_max = center_lat + radius_deg

            # Debugging: Print calculated values
            logging.debug(f"PEAKID: {file_name}")
            logging.debug(f"Center Lat: {center_lat}, Center Lon: {center_lon}")
            logging.debug(f"Radius (degrees): {radius_deg}")
            logging.debug(f"Bounding Box: {x_min}, {y_min}, {x_max}, {y_max}")

            # Prepare API parameters for image download (using bounding box coordinates)
            params = {
                "minlatitude": y_min,
                "maxlatitude": y_max,
                "minlongitude": x_min,
                "maxlongitude": x_max,
                "width": tile_pixels,
                "mask": "false",
                "download": "true",
            }

            # Construct file paths
            bbox_file_path = os.path.join(bbox_folder, f"{file_name}_1.png")  # Save bbox-specific image

            # Download the image for the bounding box
            logging.info(f"Downloading bounding box image for PEAKID {file_name}...")
            response = requests.get(api_base_url, params=params)

            if response.status_code == 200:
                # Save the bounding box image
                with open(bbox_file_path, "wb") as file:
                    file.write(response.content)
                logging.info(f"Bounding box image saved: {bbox_file_path}")

                # Append bounding box data
                bbox_data.append([file_name, x_min, y_min, x_max, y_max])
            else:
                logging.error(f"Failed to download bounding box for PEAKID {file_name} - HTTP {response.status_code}")
                retries += 1
                time.sleep(2)  # Wait before retrying
                continue

            break  # Break the retry loop if successful

        except Exception as e:
            logging.error(f"Error processing row {index}: {e}")
            retries += 1
            time.sleep(2)  # Wait before retrying
            continue

# Save bounding box data to CSV
bbox_df = pd.DataFrame(bbox_data, columns=["PEAKID", "x_min", "y_min", "x_max", "y_max"])
bbox_df.to_csv(bbox_file, index=False)
logging.info(f"Bounding box data saved to {bbox_file}")

logging.info("Process complete.")


2024-12-22 19:36:05,026 - INFO - Loaded 158 PEAKIDs to download from missing in pixel list.txt.
2024-12-22 19:36:05,223 - INFO - Filtered 158 records from seamounts.csv.
2024-12-22 19:36:05,224 - INFO - Downloading bounding box image for PEAKID 336209...
2024-12-22 19:36:06,718 - INFO - Bounding box image saved: seamounts_bboxes\336209_1.png
2024-12-22 19:36:06,719 - INFO - Downloading bounding box image for PEAKID 360059...
2024-12-22 19:36:08,158 - INFO - Bounding box image saved: seamounts_bboxes\360059_1.png
2024-12-22 19:36:08,159 - INFO - Downloading bounding box image for PEAKID 1001431...
2024-12-22 19:36:09,622 - INFO - Bounding box image saved: seamounts_bboxes\1001431_1.png
2024-12-22 19:36:09,624 - INFO - Downloading bounding box image for PEAKID 1013016...
2024-12-22 19:36:10,970 - INFO - Bounding box image saved: seamounts_bboxes\1013016_1.png
2024-12-22 19:36:10,971 - INFO - Downloading bounding box image for PEAKID 1025041...
2024-12-22 19:36:12,322 - INFO - Bounding bo

In [3]:
import os

# Path to the folder where the files are located
folder_path = 'seamounts_bboxes'

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    # Create the new filename by adding .0 before every underscore
    new_filename = filename.replace('_', '.0_')

    # Full paths for the old and new filenames
    old_file = os.path.join(folder_path, filename)
    new_file = os.path.join(folder_path, new_filename)

    # Rename the file
    os.rename(old_file, new_file)

    print(f'Renamed: {filename} -> {new_filename}')


Renamed: 1001431_1.png -> 1001431.0_1.png
Renamed: 1013016_1.png -> 1013016.0_1.png
Renamed: 1025041_1.png -> 1025041.0_1.png
Renamed: 2002867_1.png -> 2002867.0_1.png
Renamed: 2076096_1.png -> 2076096.0_1.png
Renamed: 2120674_1.png -> 2120674.0_1.png
Renamed: 2203770_1.png -> 2203770.0_1.png
Renamed: 2325818_1.png -> 2325818.0_1.png
Renamed: 2327372_1.png -> 2327372.0_1.png
Renamed: 2358338_1.png -> 2358338.0_1.png
Renamed: 2392192_1.png -> 2392192.0_1.png
Renamed: 2392966_1.png -> 2392966.0_1.png
Renamed: 2394421_1.png -> 2394421.0_1.png
Renamed: 2395572_1.png -> 2395572.0_1.png
Renamed: 2401816_1.png -> 2401816.0_1.png
Renamed: 2414507_1.png -> 2414507.0_1.png
Renamed: 2419181_1.png -> 2419181.0_1.png
Renamed: 2438586_1.png -> 2438586.0_1.png
Renamed: 2451447_1.png -> 2451447.0_1.png
Renamed: 2474642_1.png -> 2474642.0_1.png
Renamed: 2486137_1.png -> 2486137.0_1.png
Renamed: 2554156_1.png -> 2554156.0_1.png
Renamed: 2581063_1.png -> 2581063.0_1.png
Renamed: 2593656_1.png -> 2593656.